<a href="https://colab.research.google.com/github/grania38/Data-Science-Projects/blob/main/IndabaXhackathon2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,StratifiedKFold, GridSearchCV
import  sklearn.metrics as skmetr

import gc, requests
from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

In [151]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [152]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')
# some sesquences have more than 1 embedding so we need to groupby by sequence and aggregate by mean

embedding_cls = pd.read_csv('/content/drive/MyDrive/sequence_esm1b_cls_embeddings_df.csv').groupby('sequence').mean().reset_index() 
embedding_mean = pd.read_csv('/content/drive/MyDrive/sequence_esm1b_mean_embeddings_df.csv').groupby('sequence').mean().reset_index()

In [153]:
LE = LabelEncoder()
embedding_cls['sequence_enc']  = LE.fit_transform(embedding_cls['sequence'])
embedding_mean['sequence_enc'] = LE.transform(embedding_mean['sequence'])

train['sequence_enc']          = LE.transform(train['sequence'])
test['sequence_enc']           = LE.transform(test['sequence'])

In [154]:
train_cls = pd.merge(train,embedding_cls.drop('sequence',1),on='sequence_enc',how='left')
test_cls  = pd.merge(test,embedding_cls.drop('sequence',1),on='sequence_enc',how='left')

train_mean = pd.merge(train,embedding_mean.drop('sequence',1),on='sequence_enc',how='left')
test_mean  = pd.merge(test,embedding_mean.drop('sequence',1),on='sequence_enc',how='left')

In [155]:
n_splits = 10
def kfold_split(Train,y):
    Train["folds"]=-1
    kf = skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    for fold, (_, val_index) in enumerate(kf.split(Train,y)):
            Train.loc[val_index, "folds"] = fold
    return Train

target_cols = ['label']
train_mean = kfold_split(train_mean,train_mean[target_cols])
train_cls = kfold_split(train_cls,train_cls[target_cols])

In [156]:
to_drop = ['ID', 'sequence', 'sequence_enc',]
target_cols = ['label']
data2_tr  = train_mean.drop(to_drop+target_cols,1)
data2_tst = test_mean.drop(to_drop,1)

In [171]:

# modeling step 
import lightgbm as lgb
Test = data2_tst.copy()
y = train_mean["label"].values
from sklearn.metrics import accuracy_score
for fold in range(n_splits):
  print(f"#########################  Fold {fold+1}/{n_splits}  #########################")
  train_df = data2_tr[data2_tr['folds']!=fold]
  valid_df = data2_tr[data2_tr['folds']==fold]

  train_df_index = data2_tr[data2_tr['folds']!=fold].index
  valid_df_index = data2_tr[data2_tr['folds']==fold].index

  X_train = train_df.drop('folds',1)
  X_test  = valid_df.drop('folds',1)

  y_train=y[train_df_index]
  y_test=y[valid_df_index]
      
  clf = lgb.LGBMClassifier(boosting_type='dart', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100)
  clf.fit(X_train,y_train)
  #print(clf.score(X_train, y_train))
  test_preds = clf.predict_proba(Test)[:,1]
  pred = clf.predict(X_test)
  print(accuracy_score(y_test, pred))
  # param_grid = {
  #     'learning_rate': [0.005, 0.01],
  #     'n_estimators': [8,16,24],
  #     'num_leaves': [6,8,12,16], # large num_leaves helps improve accuracy but might lead to over-fitting
  #     'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
  #     'objective' : ['binary'],
  #     'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
  #     'random_state' : [500],
  #     'colsample_bytree' : [0.64, 0.65, 0.66],
  #     'subsample' : [0.7,0.75],
  #     'reg_alpha' : [1,1.2],
  #     'reg_lambda' : [1,1.2,1.4],
  #     }
  # grid_search = GridSearchCV(estimator=lgb.LGBMClassifier(), param_grid=param_grid)
  # grid_search.fit(X_train,y_train)
  # print(param_grid.best_params_)
  break 

#########################  Fold 1/10  #########################
0.9949367088607595


In [164]:
test_ = test[['ID']].copy()
test_["label"] = test_preds

In [165]:
test_.head()
test_.to_csv('Starter_notebook.csv',index=False)